# DataJoint U24 - Workflow DeepLabCut

## Workflow Automation

In the previous notebook [03-Process](./03-Process.ipynb), we ran through the workflow in detailed steps. For daily running routines, the current notebook provides a more succinct and automatic approach to run through the pipeline using some utility functions in the workflow.

The commands here run a workflow using [example data](https://downgit.github.io/#/home?url=https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) from the [00-DownloadData](./00-DataDownload_Optional.ipynb) notebook, but note where placeholders could be changed for a different dataset.

In [1]:
import os; from pathlib import Path
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-deeplabcut', ("Please move to the "
                                                              + "workflow directory")
from workflow_deeplabcut.pipeline import lab, subject, session, train, model
from workflow_deeplabcut import process

Connecting cbroz@dss-db.datajoint.io:3306


We'll be using the `process.py` script to automatically loop through all `make` functions, as a shortcut for calling each individually.

If you previously completed the [03-Process notebook](./03-Process.ipynb), you may want to delete the contents ingested there, to avoid duplication errors.

In [ ]:
safemode=None # Set to false to turn off confirmation prompts
(session.Session & 'subject="subject6"').delete(safemode=safemode)
train.TrainingParamSet.delete(safemode=safemode)
train.VideoSet.delete(safemode=safemode)

## Ingestion of subjects, sessions, videos and training parameters

Refer to the `user_data` folder in the workflow.

1. Fill subject and session information in files `subjects.csv` and `sessions.csv`
2. Fill in recording and parameter information in `recordings.csv` and `config_params.csv`
    + Add both training and estimation videos to the recording list
    + Additional columns in `config_params.csv` will be treated as model training parameters
3. Run automatic scripts prepared in `workflow_deeplabcut.ingest` for ingestion: 
    + `ingest_subjects` for `subject.Subject`
    + `ingest_sessions` - for session tables `Session`, `SessionDirectory`, and `SessionNote`
    + `ingest_dlc_items` - for ...
        - `train.ModelTrainingParamSet`
        - `train.VideoSet` and the corresponding `File` part table
        - `model.VideoRecording` and the corresponding `File` part table

In [2]:
from workflow_deeplabcut.ingest import ingest_subjects, ingest_sessions, ingest_dlc_items
ingest_subjects(); ingest_sessions(); ingest_dlc_items()


---- Inserting 0 entry(s) into subject ----

---- Inserting 2 entry(s) into session ----

---- Inserting 2 entry(s) into session_directory ----

---- Inserting 2 entry(s) into session_note ----

---- Inserting 3 entry(s) into #model_training_param_set ----

---- Inserting 2 entry(s) into video_set ----

---- Inserting 8 entry(s) into video_set__file ----

---- Inserting 2 entry(s) into video_recording ----

---- Inserting 2 entry(s) into video_recording__file ----


## Setting project variables

1. Set your root directory in your DataJoint config file, under `custom` as `dlc_root_data_dir`. 

In [4]:
import datajoint as dj; dj.config.load('dj_local_conf.json')
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
                          'openfield-Pranav-2018-10-30')            # DLC project dir
config_path = (data_dir / 'config.yaml')

2. For the purposes of this demo, we will
   1. ask DeepLabCut to structure the demo config file with `load_demo_data`. If you already did this in the [00-DataDownload notebook](./00-DataDownload_Optional.ipynb), skip this step.
   2. generate a copy to show pose estimation. This is `recording_id` 2 in `recordings.csv`. If you already did this in the [00-DataDownload notebook](./00-DataDownload_Optional.ipynb), skip this step.

In [4]:
# A
from deeplabcut.create_project.demo_data import load_demo_data
# load_demo_data(config_path)
# B
vid_path = str(data_dir).replace(" ", "\ ") + '/videos/m3v1mp4'
cmd = (f'ffmpeg -n -hide_banner -loglevel error -ss 0 -t 2 -i {vid_path}.mp4 -vcodec copy '
       + f'-acodec copy {vid_path}-copy.mp4') # New video copy, first 2 seconds
os.system(cmd)

File '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4-copy.mp4' already exists. Exiting.


256

3. Next, we pair training files with training parameters, and launch training via `process`. 
   - Some tables may try to populate without upstream keys. 
   - Others, like `RecordingInfo` already have keys loaded.
   - Note: DLC's model processes (e.g., Training, Evaluation) log a lot of information to the console, to quiet this, pass `verbose=False` to `process`

In [ ]:
key={'paramset_idx':1,'training_id':1,'video_set_id':1, 
     'project_path':'openfield-Pranav-2018-10-30/'}
train.TrainingTask.insert1(key, skip_duplicates=True)
process.run(verbose=True)
model.RecordingInfo()

4. Now to add such an upstream key: a model to the `Model` table, and `process` to evaluate.
   - Include a user-friendly `model_name`
   - Include the relative path for the project's `config.yaml`
   - Add `shuffle` and `trainingsetindex`
   - `insert_new_model` will prompt before inserting, but this can be skipped with `prompt=False`

In [ ]:
model.Model.insert_new_model(model_name='OpenField-5', 
                             dlc_config=config_path,
                             shuffle=1,
                             trainingsetindex=0,
                             paramset_idx=1, 
                             prompt=True, # True is the default behavior
                             model_description='Open field model trained 5 iterations')
process.run()

5. Add a pose estimation task, and launch via `process`.
   - Get all primary key information for a given recording
   - Add the model and `task_mode` (i.e., load vs. trigger) to be applied
   - Add any additional analysis parameters for `deeplabcut.analyze_videos`

In [ ]:
key=(model.VideoRecording & 'recording_id=2').fetch1('KEY')
key.update({'model_name': 'OpenField-5', 'task_mode': 'trigger'})
analyze_params={'save_as_csv':True} # add any others from deeplabcut.analyze_videos
model.PoseEstimationTask.insert_estimation_task(key,params=analyze_params)
process.run()

6. Retrieve estimated position data.

In [7]:
model.PoseEstimation.get_trajectory(key)

scorer    OpenField-5                                                       \
bodyparts     leftear                            rightear                    
coords              x         y    z likelihood         x           y    z   
0          125.213768  0.464425  0.0   0.142836  1.902155  184.619431  0.0   
1          125.009758  1.058969  0.0   0.136179  1.532405  183.668121  0.0   
2          123.785698  1.801253  0.0   0.150994  1.467412  183.721542  0.0   
3          122.621880  2.729937  0.0   0.150831  1.424251  184.009323  0.0   
4          123.729645  2.901060  0.0   0.163442  1.417472  183.914078  0.0   
..                ...       ...  ...        ...       ...         ...  ...   
58         240.315948 -1.135241  0.0   0.141477  2.564324  153.450378  0.0   
59         240.919571 -1.104096  0.0   0.122847  6.900490   -0.243096  0.0   
60         255.197067 -0.876162  0.0   0.141331  3.224912  170.105179  0.0   
61         255.042603  0.554700  0.0   0.152119  6.523534   -0.563077  0.0   
62         255.079330 -0.326163  0.0   0.168699  3.389258  170.141495  0.0   

scorer                                                               \
bodyparts                snout                             tailbase   
coords    likelihood         x          y    z likelihood         x   
0           0.123875 -7.285146  61.402088  0.0   0.267532  2.360505   
1           0.130291 -7.269304  61.589397  0.0   0.269269  9.910207   
2           0.129725 -6.988381  61.624317  0.0   0.266620  2.753981   
3           0.133028 -7.054953  61.331848  0.0   0.286876  2.399938   
4           0.129994 -6.633567  60.880890  0.0   0.283661  2.544708   
..               ...       ...        ...  ...        ...       ...   
58          0.108718 -6.014613  59.291553  0.0   0.264213  2.494397   
59          0.104687 -6.632689  59.683407  0.0   0.236766  3.034356   
60          0.102174 -6.597838  59.643513  0.0   0.236705  2.666216   
61          0.102816 -6.134833  59.962490  0.0   0.249565  2.555799   
62          0.103460 -6.661276  59.593884  0.0   0.236305  2.978589   

scorer                                 
bodyparts                              
coords              y    z likelihood  
0           30.929823  0.0   0.132607  
1          168.702576  0.0   0.140683  
2           30.949059  0.0   0.136884  
3           30.467714  0.0   0.146240  
4           30.362843  0.0   0.139938  
..                ...  ...        ...  
58          30.713549  0.0   0.127640  
59          30.454117  0.0   0.127521  
60          30.185883  0.0   0.123881  
61          30.326237  0.0   0.130592  
62          30.103178  0.0   0.124924  

[63 rows x 16 columns]

## Summary and next step

+ This notebook runs through the workflow in an automatic manner.

+ The next notebook [06-Drop](06-Drop_Optional.ipynb) shows how to drop schemas and tables if needed.